In [0]:
%sql
/****************************************************************/
/* SCHEMAS                                                      */
/****************************************************************/

CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.control;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.bronze;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.bronze_rejected;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.silver;
CREATE SCHEMA IF NOT EXISTS `latam-md-finance`.silver_rejected;

**CONTROL**

In [0]:
%sql
--Register bucket as external location

CREATE EXTERNAL LOCATION `latam-md-finance-control`
URL 's3://latam-md-finance-control/'
WITH (STORAGE CREDENTIAL `ibs-external-storage_awsiamrole_1749223868654`);


In [0]:
%sql
--Register bucket as external location

CREATE EXTERNAL LOCATION `latam-md-finance-silver-rejected`
URL 's3://latam-md-finance-silver-rejected/'
WITH (STORAGE CREDENTIAL `ibs-external-storage_awsiamrole_1749223868654`);


In [0]:
%sql

CREATE TABLE `latam-md-finance`.control.sys_process_source (
  process_source_id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Process Source id (Autonumeric)',
  process_source_name STRING NOT NULL COMMENT 'Process Name (Unique)',
  source_system_code STRING NOT NULL COMMENT 'Source System Code',
  process_step_name STRING NOT NULL COMMENT 'Process Step Name',
  source_sharepoint_host_name STRING COMMENT 'Sharepoint hostname',
  source_sharepoint_site_relative STRING COMMENT 'Sharepoint site relative',
  source_sharepoint_drive STRING COMMENT 'Sharepoint drive',
  source_sharepoint_file_path STRING COMMENT 'Sharepoint file path inside drive',
  source_bucket_name STRING COMMENT 'Source Bucket Name',
  source_bucket_folder_key STRING COMMENT 'Source Folder path inside bucket',
  target_bucket_name STRING COMMENT 'Target Bucket Name',
  target_bucket_folder_key STRING COMMENT 'Target Folder path inside bucket',
  archive_bucket_name STRING COMMENT 'Archive Bucket Name',
  archive_bucket_folder_key STRING COMMENT 'Archive Folder path inside bucket',
  sys_status_code STRING DEFAULT 'A' NOT NULL COMMENT 'System Row Status Code',
  sys_created_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Created by Name',
  sys_created_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Created on',
  sys_modified_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Modified by Name',
  sys_modified_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Modified on'
)
USING DELTA
LOCATION 's3://latam-md-finance-control/sys_process_source'
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql

insert into `latam-md-finance`.control.sys_process_source(
     process_source_name
    ,source_system_code
    ,process_step_name
    ,source_sharepoint_host_name
    ,source_sharepoint_site_relative
    ,source_sharepoint_drive
    ,source_sharepoint_file_path
    ,target_bucket_name
    ,target_bucket_folder_key
)
values(
    'Load BI OVC' process_source_name
    ,'sapbr' source_system_code
    ,'sharepoint to s3' process_step_name
    ,'insightabs.sharepoint.com' source_sharepoint_host_name
    ,'sites/InsightTeam' source_sharepoint_site_relative
    ,'Documents' source_sharepoint_drive
    ,'BI_OVC_TST/FIN_VARIABLE' source_sharepoint_file_path
    ,'latam-md-finance-raw' target_bucket_name
    ,'tb_fin_variable_cost_act' target_bucket_folder_key
   );


In [0]:
%sql

insert into `latam-md-finance`.control.sys_process_source(
     process_source_name
    ,source_system_code
    ,process_step_name
    ,source_bucket_name
    ,source_bucket_folder_key
    ,target_bucket_name
    ,target_bucket_folder_key
    ,archive_bucket_name
    ,archive_bucket_folder_key
)
values(
    'Load BI OVC' process_source_name
    ,'sapbr' source_system_code
    ,'raw to bronze' process_step_name
    ,'latam-md-finance-raw' source_bucket_name
    ,'tb_fin_variable_cost_act' source_bucket_folder_key
    ,'latam-md-finance-bronze' target_bucket_name
    ,'tb_fin_variable_cost_act' target_bucket_folder_key
    ,'latam-md-finance-archive' archive_bucket_name
    ,'tb_fin_variable_cost_act_sapbr' archive_bucket_folder_key
   );


In [0]:
%sql

insert into `latam-md-finance`.control.sys_process_setup(
    process_setup_name
    ,process_setup_step_number
    ,process_setup_step_name
    ,source_system_code
    ,process_setup_source_layer
    ,process_setup_source_bucket_name
    ,process_setup_source_bucket_folder_key
    ,process_setup_source_table_name
    ,process_setup_source_table_schema
    ,process_setup_source_table_catalog
    ,process_setup_source_data_definition
    ,process_setup_target_layer
    ,process_setup_target_table_name
    ,process_setup_target_table_schema
    ,process_setup_target_table_catalog
    ,process_setup_target_data_definition
    ,process_setup_target_bucket_name
    ,process_setup_target_bucket_folder_key
)
values(
    'Load BI OVC' process_setup_name
    ,3 process_setup_step_number
    ,'bronze to silver' process_setup_step_name
    ,'sapbr' source_system_code
    ,'bronze' process_setup_source_layer
    ,'latam-md-finance-bronze' process_setup_source_bucket_name
    ,'tb_fin_variable_cost_act_sapbr' process_setup_source_bucket_folder_key
    ,'tb_fin_variable_cost_act_sapbr' process_setup_source_table_name
    ,'bronze' process_setup_source_table_schema
    ,'latam-md-finance' process_setup_source_table_catalog
    ,'fin-act-bronze-sapbr' process_setup_source_data_definition
    ,'silver' process_setup_target_layer
    ,'tb_fin_variable_cost_act' process_setup_target_table_name
    ,'silver' process_setup_target_table_schema
    ,'latam-md-finance' process_setup_target_table_catalog
    ,'fin-act-silver' process_setup_target_data_definition
    ,'latam-md-finance-silver' process_setup_target_bucket_name
    ,'tb_fin_variable_cost_act' process_setup_target_bucket_folder_key
   );


In [0]:
%sql
--truncate table `latam-md-finance`.control.log_process_run

In [0]:
%sql

CREATE TABLE `latam-md-finance`.control.log_process_run (
    process_run_id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Process Run ID Number (Autonumeric)',
    fk_process_setup_id INT COMMENT 'Process Setup id (from sys_process_setup table)',
    process_run_cycle_id INT COMMENT 'Data cycle',
    process_run_source_file_name STRING COMMENT 'File name of file loaded from source (can be more than one)',
    process_run_source_record_count INT COMMENT 'Count of Records from source',
    process_run_target_record_count INT COMMENT 'Count of Records loaded in target',
    process_run_extract_begin_date STRING COMMENT 'First date of the main extract date',
    process_run_extract_end_date STRING COMMENT 'Last date of the main extract date',
    process_run_start_date TIMESTAMP COMMENT 'When process start running',
    process_run_end_date TIMESTAMP COMMENT 'When process end running',
    process_run_last_flag STRING COMMENT 'Active only for the last running',
    sys_status_code STRING DEFAULT 'A' NOT NULL COMMENT 'System Row Status Code',
    sys_created_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Created by Name',
    sys_created_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Created on',
    sys_modified_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Modifed by Name',
    sys_modified_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Modified on'
)
USING DELTA
LOCATION 's3://latam-md-finance-control/log_process_run'
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
#dbutils.fs.rm("s3://latam-md-finance-control/object_validation", recurse=True)

In [0]:
%sql

CREATE TABLE `latam-md-finance`.control.object_validation (
    object_validation_id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Object Validation ID Number (Autonumeric)',
    fk_process_setup_id INTEGER NOT NULL COMMENT 'Process Setup id (from sys_process_setup table)',
    object_validation_type STRING NOT NULL COMMENT 'Object Validation Type (PH, EB)',
    object_type STRING COMMENT 'Object Type (data, file, table…)',
    object_validation_description STRING NOT NULL COMMENT 'Object Validation Description',
    object_name STRING NOT NULL COMMENT 'Object name to be validated',
    validation_rule_code STRING COMMENT 'Validation rule Code (date_format, null, etc)',
    validation_rule_detail STRING COMMENT 'Validation rule detail (expressions or values necessary for validation)',
    master_table_name STRING COMMENT 'For Business Validations table to validate against with',
    reject_all INT DEFAULT 0 NOT NULL COMMENT 'If not pass this validation reject all records',
    sys_status_code STRING DEFAULT 'A' NOT NULL COMMENT 'System Row Status Code',
    sys_created_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Created by Name',
    sys_created_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Created on',
    sys_modified_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Modifed by Name',
    sys_modified_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Modified on'
)
USING DELTA
LOCATION 's3://latam-md-finance-control/object_validation'
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

In [0]:
%sql
truncate table `latam-md-finance`.control.object_validation;
insert into `latam-md-finance`.control.object_validation(
     fk_process_setup_id
    ,object_validation_type
    ,object_type
    ,object_validation_description
    ,object_name
    ,validation_rule_code
    ,validation_rule_detail
    ,master_table_name
    ,reject_all
    ,sys_status_code
)
values(2,'PH','data','Invalid DataType','POSTING_DATE', 'date_format','yyyyMMdd',null,1,'A')
,(2,'PH','data','Missing Value','POSTING_DATE','not_null',null,null,1,'A')
,(2,'PH','data','Missing Value','SOURCE_SYSTEM_CODE','not_null',null,null,1,'A')
,(2,'PH','data','Invalid DataType','EXPENSE_AMOUNT_LC','numeric_format','decimal(15,5)',null,0,'A')
,(2,'PH','data','Invalid DataType','POSTING_DOC_LINE_NUMBER','numeric_format','integer',null,0,'A')
,(2,'PH','data','Missing Value','POSTING_DOC_LINE_NUMBER','not_null',null,null,0,'A')
,(2,'PH','data','Value Out of Range','POSTING_DATE','out_of_range','< ORIG_DOC_ENTRY_DATE',null,0,'R')
,(2,'PH','data','Value Out of Range','POSTING_DOC_TYPE','conditional_fail',"POSTING_DOC_TYPE == 'KS' AND POSTING_DATE == ORIG_DOC_ENTRY_DATE",null,0,'A')
--para cross reference en rule_detail se pasa una lista de tuplas con los campos a comparar (esto es por si no se llaman igual en ambas tablas, la tupla es (source, target))
,(3,'BS','data','Missing Cross Reference','VENDOR_CODE','cross_reference','[("COMPANY_CODE", "COMPANY_CODE"),("VENDOR_CODE","VDR_LD_CD")]','mdo.dev.ldt_vendor',0,'A')
,(3,'BS','data','Missing Cross Reference','COMPANY_CODE','cross_reference','[("COMPANY_CODE", "COMPANY_CODE")]','mdo.dev.gdt_company',0,'A')


In [0]:
%sql

CREATE TABLE `latam-md-finance`.control.object_validation_error (
  object_validation_error_id BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Object Validation Error ID (Autonumeric)',
  fk_process_run_id INT NOT NULL COMMENT 'Process Run id (from log_process_run table)',
  fk_object_validation_id INT NOT NULL COMMENT 'Object Validation ID (from object_validation table)',
  fk_dw_row_id	STRING COMMENT 'ID of row in source table',
  object_validation_error_value STRING NOT NULL COMMENT 'Incorrect value in table/file',
  sys_status_code STRING DEFAULT 'A' NOT NULL COMMENT 'System Row Status Code',
  sys_created_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Created by Name',
  sys_created_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Created on',
  sys_modified_by_name STRING DEFAULT 'MANUAL INSERT' NOT NULL COMMENT 'System Modified by Name',
  sys_modified_on TIMESTAMP DEFAULT current_timestamp() NOT NULL COMMENT 'System Modified on'
)
USING DELTA
LOCATION 's3://latam-md-finance-control/object_validation_error'
TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

**FINANCE**

In [0]:
#CREATE TABLE - Bronze - tb_fin_variable_cost_act
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

schema = StructType([
    StructField("SOURCE_SYSTEM_CODE", StringType(), nullable=True),
    StructField("COMPANY_CODE", StringType(), nullable=True),
    StructField("POSTING_DATE", StringType(), nullable=True),
    StructField("POSTING_DOC_TYPE", StringType(), nullable=True),
    StructField("POSTING_DOC_NUMBER", StringType(), nullable=True),
    StructField("POSTING_DOC_LINE_NUMBER", StringType(), nullable=True),
    StructField("POSTING_TEXT", StringType(), nullable=True),
    StructField("ORIG_DOC_ENTRY_DATE", StringType(), nullable=True),
    StructField("ORIG_DOC_ISSUE_DATE", StringType(), nullable=True),
    StructField("REVERSE_POSTING_DOC_NUMBER", StringType(), nullable=True),
    StructField("COST_CENTER_CODE", StringType(), nullable=True),
    StructField("SAP_ACCOUNT_CODE", StringType(), nullable=True),
    StructField("SAP_ACCOUNT_NAME", StringType(), nullable=True),
    StructField("OFF_SAP_ACCOUNT_CODE", StringType(), nullable=True),
    StructField("OFF_SAP_ACCOUNT_NAME", StringType(), nullable=True),
    StructField("TRANS_CURRENCY_CODE", StringType(), nullable=True),
    StructField("LOCAL_CURRENCY_CODE", StringType(), nullable=True),
    StructField("USER_NAME", StringType(), nullable=True),
    StructField("EXPENSE_AMOUNT_LC", StringType(), nullable=True),
    StructField("EXPENSE_AMOUNT_TC", StringType(), nullable=True),
    StructField("TRANS_UM_CODE", StringType(), nullable=True),
    StructField("EXPENSE_QTY_TRANS_UM", StringType(), nullable=True),
    StructField("VENDOR_CODE", StringType(), nullable=True),
    StructField("VENDOR_NAME", StringType(), nullable=True),
    StructField("COST_CENTER_NAME", StringType(), nullable=True),
    StructField("CCT_TYPE_LD_CD", StringType(), nullable=True),
    StructField("DW_ROW_STAT_CD", StringType()),
    StructField("DW_ISRT_TS", TimestampType()),
    StructField("DW_UPDT_TS", TimestampType())
])

# create empty table
empty_df = spark.createDataFrame([], schema)
empty_df.write.format("delta").mode("overwrite").save('s3://latam-md-finance-bronze/tb_fin_variable_cost_act')



In [0]:
table = "`latam-md-finance`.bronze.tb_fin_variable_cost_act"
path = "s3://latam-md-finance-bronze/tb_fin_variable_cost_act"
spark.sql("CREATE TABLE IF NOT EXISTS " + table + " USING DELTA LOCATION '" +  path + "'")


In [0]:
%sql 
--Add comments & defaults
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_ROW_STAT_CD SET DEFAULT 'A';

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_ISRT_TS SET DEFAULT getdate();

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act
ALTER COLUMN DW_UPDT_TS SET DEFAULT getdate();


ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SOURCE_SYSTEM_CODE SOURCE_SYSTEM_CODE STRING COMMENT 'Local System Code where the data have been extracted (it could be used to differentiate potential equal code with different meaning). Possible values: SAPBR, SAP1C';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COMPANY_CODE COMPANY_CODE STRING COMMENT 'Company Code according to Prysmian Group';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DATE POSTING_DATE STRING COMMENT 'Posting Date → Reference date (both for ETL and Reporting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_TYPE POSTING_DOC_TYPE STRING COMMENT 'Posting Document Type';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_NUMBER POSTING_DOC_NUMBER STRING COMMENT 'Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_DOC_LINE_NUMBER POSTING_DOC_LINE_NUMBER STRING COMMENT 'Posting Document Line Number';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN POSTING_TEXT POSTING_TEXT STRING COMMENT 'Posting Text';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN ORIG_DOC_ENTRY_DATE ORIG_DOC_ENTRY_DATE STRING COMMENT 'Document Entry Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN ORIG_DOC_ISSUE_DATE ORIG_DOC_ISSUE_DATE STRING COMMENT 'Document Issue Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN REVERSE_POSTING_DOC_NUMBER REVERSE_POSTING_DOC_NUMBER STRING COMMENT 'Reverse Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COST_CENTER_CODE COST_CENTER_CODE STRING COMMENT 'Cost Center Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SAP_ACCOUNT_CODE SAP_ACCOUNT_CODE STRING COMMENT 'SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN SAP_ACCOUNT_NAME SAP_ACCOUNT_NAME STRING COMMENT 'SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN OFF_SAP_ACCOUNT_CODE OFF_SAP_ACCOUNT_CODE STRING COMMENT 'Offsetting SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN OFF_SAP_ACCOUNT_NAME OFF_SAP_ACCOUNT_NAME STRING COMMENT 'Offsetting SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN TRANS_CURRENCY_CODE TRANS_CURRENCY_CODE STRING COMMENT 'Transaction Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN LOCAL_CURRENCY_CODE LOCAL_CURRENCY_CODE STRING COMMENT 'Local Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN USER_NAME USER_NAME STRING COMMENT 'User Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_AMOUNT_LC EXPENSE_AMOUNT_LC STRING COMMENT 'Expense in Local Currency (Value/Obj. Crcy)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_AMOUNT_TC EXPENSE_AMOUNT_TC STRING COMMENT 'Expense in Transaction Currency';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN TRANS_UM_CODE TRANS_UM_CODE STRING COMMENT 'Transaction Unit of Measure Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN EXPENSE_QTY_TRANS_UM EXPENSE_QTY_TRANS_UM STRING COMMENT 'Good Receipt Quantity Stocking (Base) UoM';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN VENDOR_CODE VENDOR_CODE STRING COMMENT 'Vendor Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN VENDOR_NAME VENDOR_NAME STRING COMMENT 'Master information: Vendor Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN COST_CENTER_NAME COST_CENTER_NAME STRING COMMENT 'Cost Center Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN CCT_TYPE_LD_CD CCT_TYPE_LD_CD STRING COMMENT 'Master information: Cost Center Type Code → It indicates which ones have been included in this extract';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_ROW_STAT_CD DW_ROW_STAT_CD STRING COMMENT 'Row status';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_ISRT_TS DW_ISRT_TS TIMESTAMP COMMENT 'Insert Row timestamp';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act CHANGE COLUMN DW_UPDT_TS DW_UPDT_TS TIMESTAMP COMMENT 'Update Row timestamp';



In [0]:
%sql 

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN SOURCE_SYSTEM_CODE STRING COMMENT 'Local System Code where the data have been extracted (it could be used to differentiate potential equal code with different meaning). Possible values: SAPBR, SAP1C';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN COMPANY_CODE STRING COMMENT 'Company Code according to Prysmian Group';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN POSTING_DATE STRING COMMENT 'Posting Date → Reference date (both for ETL and Reporting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN POSTING_DOC_TYPE STRING COMMENT 'Posting Document Type';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN POSTING_DOC_NUMBER STRING COMMENT 'Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN POSTING_DOC_LINE_NUMBER STRING COMMENT 'Posting Document Line Number';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN POSTING_TEXT STRING COMMENT 'Posting Text';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN ORIG_DOC_ENTRY_DATE STRING COMMENT 'Document Entry Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN ORIG_DOC_ISSUE_DATE STRING COMMENT 'Document Issue Date';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN REVERSE_POSTING_DOC_NUMBER STRING COMMENT 'Reverse Reference Document Number (Posting)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN COST_CENTER_CODE STRING COMMENT 'Cost Center Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN SAP_ACCOUNT_CODE STRING COMMENT 'SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN SAP_ACCOUNT_NAME STRING COMMENT 'SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN OFF_SAP_ACCOUNT_CODE STRING COMMENT 'Offsetting SAP Account Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN OFF_SAP_ACCOUNT_NAME STRING COMMENT 'Offsetting SAP Account Description';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN TRANS_CURRENCY_CODE STRING COMMENT 'Transaction Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN LOCAL_CURRENCY_CODE STRING COMMENT 'Local Currency Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN USER_NAME STRING COMMENT 'User Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN EXPENSE_AMOUNT_LC STRING COMMENT 'Expense in Local Currency (Value/Obj. Crcy)';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN EXPENSE_AMOUNT_TC STRING COMMENT 'Expense in Transaction Currency';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN TRANS_UM_CODE STRING COMMENT 'Transaction Unit of Measure Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN EXPENSE_QTY_TRANS_UM STRING COMMENT 'Good Receipt Quantity Stocking (Base) UoM';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN VENDOR_CODE STRING COMMENT 'Vendor Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN VENDOR_NAME STRING COMMENT 'Master information: Vendor Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN COST_CENTER_NAME STRING COMMENT 'Cost Center Name';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN CCT_TYPE_LD_CD STRING COMMENT 'Master information: Cost Center Type Code → It indicates which ones have been included in this extract';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN DW_ROW_STAT_CD STRING COMMENT 'ROW Status Code';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN DW_ISRT_TS TIMESTAMP COMMENT 'Insertion Timestamp';
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr ADD COLUMN DW_UPDT_TS TIMESTAMP COMMENT 'Update';

--Add comments & defaults
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_ROW_STAT_CD SET DEFAULT 'A';

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_ISRT_TS SET DEFAULT getdate();

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_UPDT_TS SET DEFAULT getdate();




In [0]:
%sql
-- REPLACE se usa cuando la tabla ya está creada en el catalog, posteriormente se agrega una columna en el parqué (si se hace un select se ve), pero no aparece en el Catalog.
-- Hay que reemplazar todo el esquema con el nuevo y al final, hay que volver a recrear los default.
-- Se puede obtener el esquema con DESCRIBE TABLE <table name>


ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
REPLACE COLUMNS (
  SOURCE_SYSTEM_CODE STRING COMMENT 'Local System Code where the data have been extracted (it could be used to differentiate potential equal code with different meaning). Possible values: SAPBR, SAP1C',
  COMPANY_CODE STRING COMMENT 'Company Code according to Prysmian Group',
  POSTING_DATE STRING COMMENT 'Posting Date → Reference date (both for ETL and Reporting)',
  POSTING_DOC_TYPE STRING COMMENT 'Posting Document Type',
  POSTING_DOC_NUMBER STRING COMMENT 'Reference Document Number (Posting)',
  POSTING_DOC_LINE_NUMBER STRING COMMENT 'Posting Document Line Number',
  POSTING_TEXT STRING COMMENT 'Posting Text',
  ORIG_DOC_ENTRY_DATE STRING COMMENT 'Document Entry Date',
  ORIG_DOC_ISSUE_DATE STRING COMMENT 'Document Issue Date',
  REVERSE_POSTING_DOC_NUMBER STRING COMMENT 'Reverse Reference Document Number (Posting)',
  COST_CENTER_CODE STRING COMMENT 'Cost Center Code',
  SAP_ACCOUNT_CODE STRING COMMENT 'SAP Account Code',
  SAP_ACCOUNT_NAME STRING COMMENT 'SAP Account Description',
  OFF_SAP_ACCOUNT_CODE STRING COMMENT 'Offsetting SAP Account Code',
  OFF_SAP_ACCOUNT_NAME STRING COMMENT 'Offsetting SAP Account Description',
  TRANS_CURRENCY_CODE STRING COMMENT 'Transaction Currency Code',
  LOCAL_CURRENCY_CODE STRING COMMENT 'Local Currency Code',
  USER_NAME STRING COMMENT 'User Name',
  EXPENSE_AMOUNT_LC STRING COMMENT 'Expense in Local Currency (Value/Obj. Crcy)',
  EXPENSE_AMOUNT_TC STRING COMMENT 'Expense in Transaction Currency',
  TRANS_UM_CODE STRING COMMENT 'Transaction Unit of Measure Code',
  EXPENSE_QTY_TRANS_UM STRING COMMENT 'Good Receipt Quantity Stocking (Base) UoM',
  VENDOR_CODE STRING COMMENT 'Vendor Code',
  VENDOR_NAME STRING COMMENT 'Master information: Vendor Name',
  COST_CENTER_NAME STRING COMMENT 'Cost Center Name',
  CCT_TYPE_LD_CD STRING COMMENT 'Master information: Cost Center Type Code → It indicates which ones have been included in this extract',
  DW_ROW_STAT_CD STRING COMMENT 'ROW Status Code',
  DW_ISRT_TS TIMESTAMP COMMENT 'Insertion Timestamp',
  DW_UPDT_TS TIMESTAMP COMMENT 'Update',
  POSTING_TEXT_ADDITIONAL STRING,
  DW_ROW_ID STRING COMMENT 'process nbr + row id',
  DW_VALID_FROM_DT TIMESTAMP,
  DW_VALID_TO_DT TIMESTAMP,
  DW_CURR_ROW_FLG BOOLEAN,
  DW_FILE_NAME STRING
);

--Add comments & defaults
ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
SET TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported');

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_ROW_STAT_CD SET DEFAULT 'A';

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_ISRT_TS SET DEFAULT getdate();

ALTER TABLE `latam-md-finance`.bronze.tb_fin_variable_cost_act_sapbr
ALTER COLUMN DW_UPDT_TS SET DEFAULT getdate();
